<a href="https://colab.research.google.com/github/MJ199999/SW-Project/blob/master/poem_keywords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://wikidocs.net/159468



In [1]:
!pip install sentence_transformers
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 4.7 MB 17.0 MB/s 
     |████████████████████████████████| 1.3 MB 77.5 MB/s 
     |████████████████████████████████| 120 kB 94.3 MB/s 
     |████████████████████████████████| 6.6 MB 58.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c77473347024bafe85163cb3b70bbe68f4ab31af1ca5aee07529b6224a730d0f
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 7.0 MB/s 
     |████████████████████████████████| 453 kB 73.4 MB/s 


# 데이터 로드

In [1]:
import numpy as np
import itertools
import pandas as pd

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [7]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!wget https://github.com/MJ199999/SW-Project/raw/master/poem_key.xlsx

--2022-08-26 06:05:39--  https://github.com/MJ199999/SW-Project/raw/master/poem_key.xlsx
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MJ199999/SW-Project/master/poem_key.xlsx [following]
--2022-08-26 06:05:39--  https://raw.githubusercontent.com/MJ199999/SW-Project/master/poem_key.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131833 (129K) [application/octet-stream]
Saving to: ‘poem_key.xlsx.1’

poem_key.xlsx.1     100%[===================>] 128.74K  --.-KB/s    in 0.01s   

2022-08-26 06:05:39 (9.90 MB/s) - ‘poem_key.xlsx.1’ saved [131833/131833]



In [3]:
df = pd.read_excel('poem_key.xlsx')

In [4]:
df.head()

,시인,제목,시,키워드
0,강은교,우리가 물이 되어,우리가 물이 되어 만난다면\n가문 어느 집에선들 좋아하지 않으랴.\n우리가 키 큰 ...,그리움
1,김춘수,꽃을 위한 서시,나는 시방 위험한 짐승이다.\n나의 손이 닿으면 너는 \n미지의 까마득한 어둠이 된...,그리움
2,김춘수,꽃,내가 그의 이름을 불러 주기 전에는\n그는 다만\n하나의 몸짓에 지나지 않았다. \...,그리움
3,김춘수,능금,그는 그리움에 산다.\n그리움은 익어서 \n스스로도 견디기 어려운\n빛깔이 되고 향...,그리움
4,김춘수,부두에서,바다에 굽힌 사나이들\n하루의 노동을 끝낸\n저 사나이들의 억센 팔에 안긴\n깨지지...,노동


In [5]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면

    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [6]:
keyword_list1=[] # 키워드전체
keyword_list2=[] #키워드 1
keyword_list3=[] #키워드 2
keyword_list4=[] #키워드 3
keyword_list5=[] #키워드 4
keyword_list6=[] #키워드 5

In [7]:
# 시 데이터 처리 
for i in range(124, len(df)):
    poem = df['시'].iloc[i] #시 하나씩 가져오기(i번째 행의 시 가져오기 )
    okt = Okt() 

    tokenized_doc = okt.pos(poem)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun']) #명사 만들기 
    
    
    n_gram_range = (1, 1)

    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns]) #벡터화 시키기 
    candidates = count.get_feature_names_out()
    

    model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
    doc_embedding = model.encode([poem])
    candidate_embeddings = model.encode(candidates)

    print(i,'번째 keyword 벡터:',candidate_embeddings)

    keywords=mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)
    print(i,'번째:',keywords)

124 번째 keyword 벡터: [[ 0.05001755  0.01441947  1.6760863  ...  0.20538086 -0.18299681
   0.16654363]
 [ 0.3541439  -0.9159627   0.89350206 ... -0.44685683 -0.5365576
  -0.12457298]
 [ 0.01835023  0.09426823  1.763912   ...  0.1935659  -0.21955696
   0.28732896]
 ...
 [ 0.14349292  0.00886437  1.5960026  ...  0.018794   -0.134466
   0.07395958]
 [ 0.31730422 -0.2063297   1.2259587  ...  0.16608123 -0.10313053
  -0.1601667 ]
 [ 0.08377461 -0.12700231  1.9721943  ...  0.17973347 -0.06122566
   0.10019918]]
124 번째: ['옥수수', '옷가지', '서쪽', '은하', '거리']
125 번째 keyword 벡터: [[ 0.06356395 -0.02038732  1.6797994  ...  0.16571844 -0.12081926
   0.15623231]
 [ 0.19238234 -0.0437884   1.2366984  ...  0.07592324 -0.11446976
   0.05797461]
 [-0.06285686  0.05900239  1.7017878  ... -0.3519666  -0.18497108
   0.13698946]
 ...
 [-0.29922694 -0.11200915  1.4359744  ...  0.22950524 -0.02630088
   0.1574336 ]
 [ 0.48301542 -0.2626803   2.0200074  ... -0.1596535  -0.7188151
  -0.32440948]
 [ 0.03922465  0.044296

KeyboardInterrupt: ignored

In [10]:
print(doc_embedding)

[[ 1.32584795e-01  4.65722173e-01 -2.87275821e-01 -1.19323321e-01
  -2.27907434e-01 -5.53653657e-01  2.35266656e-01  3.34515065e-01
   2.66038358e-01 -9.02881771e-02 -2.67822325e-01  2.00771973e-01
  -6.99505627e-01  1.01793535e-01  2.64147520e-01  1.89431086e-01
  -7.02068117e-03 -1.76316395e-01  3.88924122e-01  5.99664524e-02
   7.56538194e-03  3.10319304e-01  2.95684904e-01 -2.85521727e-02
   1.04648554e+00 -4.31235313e-01 -3.20941836e-01  3.75867039e-01
   2.01145738e-01  1.59874067e-01  6.68606162e-02 -1.68423504e-02
   4.04083818e-01 -3.65397394e-01 -4.38518256e-01  3.21760289e-02
   9.99822989e-02 -2.94011850e-02  3.41142952e-01 -4.47842389e-01
  -1.77128822e-01  5.21000624e-01 -1.01675175e-01 -8.44228268e-01
   4.54442166e-02  4.60804045e-01 -1.12001836e+00  9.95985642e-02
   5.67891061e-01  5.54373264e-01 -3.88748124e-02  1.70275092e-01
   8.75312328e-01 -4.08772022e-01 -5.29868156e-03  5.63965626e-02
  -3.84917945e-01  1.43594265e-01 -2.82609999e-01 -2.55816668e-01
  -9.85645

In [27]:
print(keywords)

['꽃나무', '영원', '겨울', '누구', '노래']


In [28]:

print(i)

97


In [22]:
keyword_list1.append(keywords) 
keyword_list2.append(keywords[0])
keyword_list3.append(keywords[1])
keyword_list4.append(keywords[2])
keyword_list5.append(keywords[3])
keyword_list6.append(keywords[4])

df['키워드 전체'] = ""
df['키워드 전체'].iloc[0] = keyword_list1
df['키워드1'] = ""
df["키워드1"].iloc[0]=keyword_list2
df['키워드2'] = ""
df["키워드2"].iloc[0]=keyword_list3
df['키워드3'] = ""
df["키워드3"].iloc[0]=keyword_list4
df['키워드4'] = ""
df["키워드4"].iloc[0]=keyword_list5
df['키워드5'] = ""
df["키워드5"].iloc[0]=keyword_list6

In [23]:
df.head()

,시인,제목,시,키워드,키워드 전체,키워드1,키워드2,키워드3,키워드4,키워드5
0,강은교,우리가 물이 되어,우리가 물이 되어 만난다면\n가문 어느 집에선들 좋아하지 않으랴.\n우리가 키 큰 ...,그리움,"[[꽃나무, 영원, 겨울, 누구, 노래], [꽃나무, 영원, 겨울, 누구, 노래]]","[꽃나무, 꽃나무]","[영원, 영원]","[겨울, 겨울]","[누구, 누구]","[노래, 노래]"
1,김춘수,꽃을 위한 서시,나는 시방 위험한 짐승이다.\n나의 손이 닿으면 너는 \n미지의 까마득한 어둠이 된...,그리움,,,,,,
2,김춘수,꽃,내가 그의 이름을 불러 주기 전에는\n그는 다만\n하나의 몸짓에 지나지 않았다. \...,그리움,,,,,,
3,김춘수,능금,그는 그리움에 산다.\n그리움은 익어서 \n스스로도 견디기 어려운\n빛깔이 되고 향...,그리움,,,,,,
4,김춘수,부두에서,바다에 굽힌 사나이들\n하루의 노동을 끝낸\n저 사나이들의 억센 팔에 안긴\n깨지지...,노동,,,,,,
